In [1]:
import pandas as pd
import json
import re
from transformers import AutoTokenizer
pd.set_option('display.max_colwidth', None)

# Functions

In [2]:
def read_file_txt(file_path)->str:

  with open(file_path, "r", encoding="Windows-1252", errors="ignore") as file:
    content = file.read()

  return content

def count_ids(content: str) -> None:
   
    suffixes = ["Q1", "C1", "C2", 'C3', "C4"]
    counts = {}

    for suffix in suffixes:
        pattern = fr'[\'"]ID[\'"]:\s*[\'"]\d+{suffix}[\'"]' #     pattern = fr'"ID":\s*"\d+{suffix}"' pega menos id
        matches = re.findall(pattern, content)
        counts[suffix] = len(matches)
        print(f"Total de IDs com sufixo '{suffix}': {counts[suffix]}")

    total_ids = sum(counts.values())
    print("Total ids:", total_ids)

def extrair_dados_para_dataframe_by_bucket(content):
    # Dividir o conteúdo por blocos de dados
    blocos_dados = content.split('json\n[')
    blocos_dados = [bloco.replace('\u200b', '').strip() for bloco in blocos_dados]

    dados_extraidos = []

    # Função para extrair dados dos novos blocos de texto com colunas ID, PT e GLOSA
    def extrair_dados_multiplos(texto):
        try:
            # Tenta carregar o texto diretamente como JSON, caso ele já esteja em um formato válido
            dados = json.loads(texto)
            if isinstance(dados, list):  # Verifica se é uma lista de dicionários
                return [(item.get("ID"), item.get("PT"), item.get("GLOSA")) for item in dados]
        except json.JSONDecodeError:
            # Caso falhe em decodificar, utiliza regex para buscar blocos que não estão no formato JSON
            padrao = re.compile(
                r'"ID":\s*"([^"]+)"\s*,\s*"PT":\s*"(.*?)"\s*,\s*"(?:GLOSA|GLOSSARY|GLOSE)":\s*"(.*?)"',
                re.DOTALL
            )
            matches = padrao.findall(texto)
            return [(match[0], match[1], match[2] if match[2] != 'null' else None) for match in matches]

        return []

    # Extrair dados de cada bloco, ignorando o primeiro elemento vazio
    for bloco in blocos_dados[1:]:
        dados_extraidos.extend(extrair_dados_multiplos(bloco))

    # Criar o DataFrame diretamente com a lista de dados extraídos
    df = pd.DataFrame(dados_extraidos, columns=['ID', 'PT', 'GLOSA'])
    return df

def extrair_dados_para_dataframe_single_block(content):
    content = content.replace("json\n", "").strip()
    pattern = re.compile(
        r"\{\s*'ID':\s*'([^']+)',\s*'PT':\s*'([^']+)',\s*'GLOSA':\s*'([^']*)'\s*\}"
    )

    dados_extraidos = pattern.findall(content)

    df = pd.DataFrame(dados_extraidos, columns=['ID', 'PT', 'GLOSA'])
    return df

# Base de Dados

## Base

In [3]:
df_base = pd.read_csv("../DATASETS/mmlu_pro_to_gloss.csv")
df_base.head(2)

,ID,PT,TOKENS_EST_ID,TOKENS_EST_PT,TOTAL_TOKENS
0,0Q1,"O grupo simétrico $S_n$ tem $\n\\factorial{n}$ elementos, portanto, é falso que $S_{10}$ tenha 10 elementos.\nEncontre a característica do anel 2Z.",3,61,64
1,1Q1,"Seja V o conjunto de todos os polinômios reais p(x). Sejam as transformações T, S definidas em V por T:p(x) -> xp(x) e S:p(x) -> p",3,59,62


## Bucket

In [4]:
print("Bucket---")
content_bucket = read_file_txt("../DATASETS/glosa_mmlu_pro_to_gloss_response.txt")
count_ids(content_bucket)
df_bucket = extrair_dados_para_dataframe_by_bucket(content_bucket)
df_bucket.head()

Bucket---
Total de IDs com sufixo 'Q1': 10554
Total de IDs com sufixo 'C1': 11522
Total de IDs com sufixo 'C2': 11698
Total de IDs com sufixo 'C3': 11388
Total de IDs com sufixo 'C4': 11591
Total ids: 56753


,ID,PT,GLOSA
0,0Q1,"O grupo simétrico $S_n$ tem $\\n\\\\factorial{n}$ elementos, portanto, é falso que $S_{10}$ tenha 10 elementos.\\nEncontre a característica do anel 2Z.","GRUPO SIMÉTRICO S N TER FATORIAL N ELEMENTO, PORTANTO FALSO S 10 TER 10 ELEMENTO. ENCONTRAR CARACTERÍSTICA ANEL 2Z."
1,1Q1,"Seja V o conjunto de todos os polinômios reais p(x). Sejam as transformações T, S definidas em V por T:p(x) -> xp(x) e S:p(x) -> p",V CONJUNTO POLINÔMIO REAL P X. TRANSFORMAÇÃO T S DEFINIR V T P X PARA X P X S P X PARA P.
2,2Q1,"Seja A o conjunto de todos os pares ordenados de inteiros (m, n) tais que 7m + 12n = 22. Qual é o maior número negativo no conjunto B = {m + n : (m, n) \\\\em A}?",A CONJUNTO PAR ORDENADO INTEIRO M N 7M + 12N = 22. MAIOR NÚMERO NEGATIVO CONJUNTO B = M + N M N EM A?
3,3Q1,"Um tanque contém inicialmente uma solução salina de 3 gramas de sal dissolvido em 100 litros de água. Uma solução salina contendo 0,02 gramas de sal por litro de água é pulverizada no tanque a uma taxa de 4 litros por minuto. A solução pulverizada é continuamente misturada com a solução salina no tanque, e a mistura flui para fora do tanque a uma taxa de 4 litros por minuto. Se a mistura for instantânea, quantos gramas de sal estão no tanque após 100 minutos?","TANQUE CONTER SOLUÇÃO SALINA 3 GRAMA SAL DISSOLVER 100 LITRO ÁGUA. SOLUÇÃO SALINA CONTER 0.02 GRAMA SAL LITRO ÁGUA PULVERIZAR TANQUE TAXA 4 LITRO MINUTO. SOLUÇÃO PULVERIZAR MISTURAR SOLUÇÃO SALINA TANQUE, MISTURA FLUIR TANQUE TAXA 4 LITRO MINUTO. MISTURA INSTANTÂNEO, QUANTO GRAMA SAL TANQUE APÓS 100 MINUTO?"
4,4Q1,Um total de 30 jogadores jogarão basquete em um parque. Haverá exatamente 5 jogadores em cada time. Qual afirmação explica corretamente como encontrar o número de times necessários?,TOTAL 30 JOGADOR JOGAR BASQUETE PARQUE. TER 5 JOGADOR CADA TIME. AFIRMAÇÃO EXPLICAR CORRETAMENTE ENCONTRAR NÚMERO TIME NECESSÁRIO?


## Single

In [6]:
print("Single---")
content_single = read_file_txt("../DATASETS/glosa_mmlu_pro_to_gloss_single_response.txt")
count_ids(content_single)
df_single = extrair_dados_para_dataframe_single_block(content_single)
df_single.head()

Single---
Total de IDs com sufixo 'Q1': 1268
Total de IDs com sufixo 'C1': 390
Total de IDs com sufixo 'C2': 220
Total de IDs com sufixo 'C3': 528
Total de IDs com sufixo 'C4': 317
Total ids: 2723


,ID,PT,GLOSA
0,177Q1,John Wilson se aposentou aos 62 anos com ganhos médios anuais de $ 5400. Sua esposa também tinha 62 anos quando ele se aposentou. Quanto os Wilsons recebem por mês?,JOHN WILSON APOSENTAR 62 ANOS GANHO MÉDIO ANO 5400 DÓLAR ESPOSA TAMBÉM 62 ANOS ELE APOSENTAR QUANTO WILSON RECEBER MÊS?
1,181Q1,"O Sr. Casey comprou móveis de escritório por $ 725. Eles têm um valor de recuperação estimado de $ 25,00 ao final de 10 anos. Usando o método da soma dos dígitos dos anos, encontre a depreciação para o quarto ano.","SR. CASEY COMPRAR MÓVEL ESCRITÓRIO SETECENTOS E VINTE E CINCO DÓLAR. VALOR RECUPERAÇÃO VINTE E CINCO DÓLAR DEZ ANO. MÉTODO SOMA DÍGITO ANO, DEPRECIAÇÃO QUARTO ANO ENCONTRAR."
2,182Q1,"Steven Fox é um vendedor da Maxi-Power Vacuum Cleaner Co. Ele recebe $ 32,50 por cada aspirador que vende, mais uma comissão de 25% sobre todos os acessórios e suprimentos que vende. Encontre sua renda total para o mês em que ele vende 14 aspiradores e $ 237,40 em acessórios e suprimentos.","STEVEN FOX VENDEDOR MAXI-POWER ASPIRADOR ELE RECEBER 32,50 DÓLAR CADA ASPIRADOR VENDER MAIS COMISSÃO 25% ACESSÓRIOS SUPRIMENTOS VENDER ELE ENCONTRAR RENDA TOTAL MÊS ELE VENDER 14 ASPIRADOR 237,40 DÓLAR ACESSÓRIOS SUPRIMENTOS."
3,183Q1,"Um rádio que custa US$ 38,00 mais 3% de imposto especial de consumo e 4% de imposto sobre vendas pode ser comprado com uma entrada de US$ 10,00 e 3 parcelas de US$ 11,60. (A) Qual é o preço à vista do rádio? (B) Qual é o preço à vista do rádio?",RÁDIO CUSTAR US DÓLAR 38 MAIS 3 POR CENTO IMPOSTO CONSUMO ESPECIAL 4 POR CENTO IMPOSTO VENDA COMPRAR ENTRADA US DÓLAR 10 PARCELA 3 US DÓLAR 11.60 PREÇO VISTA RÁDIO QUAL?
4,185Q1,"O Sr. Langham decide tomar um empréstimo de seu banco por $60 a uma taxa nominal de 6%. Se ele concordar em pagar o empréstimo em parcelas iguais ao longo de um período de 15 meses, quanto será cada um dos pagamentos mensais?",SR. LANGHAM DECIDIR TOMAR EMPRÉSTIMO BANCO $60 TAXA 6% ACORDAR PAGAR EMPRÉSTIMO PARCELA IGUAL 15 MÊS QUANTO PAGAMENTO MENSAL?


# Join bucked and sigle

In [7]:
df_glosa = pd.concat([df_bucket, df_single])
df_glosa.head()

,ID,PT,GLOSA
0,0Q1,"O grupo simétrico $S_n$ tem $\\n\\\\factorial{n}$ elementos, portanto, é falso que $S_{10}$ tenha 10 elementos.\\nEncontre a característica do anel 2Z.","GRUPO SIMÉTRICO S N TER FATORIAL N ELEMENTO, PORTANTO FALSO S 10 TER 10 ELEMENTO. ENCONTRAR CARACTERÍSTICA ANEL 2Z."
1,1Q1,"Seja V o conjunto de todos os polinômios reais p(x). Sejam as transformações T, S definidas em V por T:p(x) -> xp(x) e S:p(x) -> p",V CONJUNTO POLINÔMIO REAL P X. TRANSFORMAÇÃO T S DEFINIR V T P X PARA X P X S P X PARA P.
2,2Q1,"Seja A o conjunto de todos os pares ordenados de inteiros (m, n) tais que 7m + 12n = 22. Qual é o maior número negativo no conjunto B = {m + n : (m, n) \\\\em A}?",A CONJUNTO PAR ORDENADO INTEIRO M N 7M + 12N = 22. MAIOR NÚMERO NEGATIVO CONJUNTO B = M + N M N EM A?
3,3Q1,"Um tanque contém inicialmente uma solução salina de 3 gramas de sal dissolvido em 100 litros de água. Uma solução salina contendo 0,02 gramas de sal por litro de água é pulverizada no tanque a uma taxa de 4 litros por minuto. A solução pulverizada é continuamente misturada com a solução salina no tanque, e a mistura flui para fora do tanque a uma taxa de 4 litros por minuto. Se a mistura for instantânea, quantos gramas de sal estão no tanque após 100 minutos?","TANQUE CONTER SOLUÇÃO SALINA 3 GRAMA SAL DISSOLVER 100 LITRO ÁGUA. SOLUÇÃO SALINA CONTER 0.02 GRAMA SAL LITRO ÁGUA PULVERIZAR TANQUE TAXA 4 LITRO MINUTO. SOLUÇÃO PULVERIZAR MISTURAR SOLUÇÃO SALINA TANQUE, MISTURA FLUIR TANQUE TAXA 4 LITRO MINUTO. MISTURA INSTANTÂNEO, QUANTO GRAMA SAL TANQUE APÓS 100 MINUTO?"
4,4Q1,Um total de 30 jogadores jogarão basquete em um parque. Haverá exatamente 5 jogadores em cada time. Qual afirmação explica corretamente como encontrar o número de times necessários?,TOTAL 30 JOGADOR JOGAR BASQUETE PARQUE. TER 5 JOGADOR CADA TIME. AFIRMAÇÃO EXPLICAR CORRETAMENTE ENCONTRAR NÚMERO TIME NECESSÁRIO?


In [8]:
df_glosa.shape

(120644, 3)

In [14]:
figurative = pd.read_csv(r"C:\Users\annap\Documents\UFLA\EVAL\DATASETS\mmlu_pro_to_figurative.csv")
figurative.head()

,ID,ORIGINAL,TOKENS_EST_ID,TOKENS_EST_ORIGINAL,TOTAL_TOKENS
0,0Q1,"O grupo simétrico $S_n$ tem $\n\factorial{n}$ elementos, portanto não é verdade que $S_{10}$ tem 10 elementos.\nEncontre a característica do anel 2Z.",3,61,64
1,1Q1,"Seja V o conjunto de todos os polinômios reais p(x). Sejam as transformações T, S definidas em V por T:p(x) -> xp(x) e S:p(x) -> p'(x) = d/dx p(x), e interprete (ST)(p(x)) como S(T(p(x))). Qual das seguintes opções é verdadeira?",3,110,113
2,2Q1,"Seja A o conjunto de todos os pares ordenados de inteiros (m, n) tais que 7m + 12n = 22. Qual é o maior número negativo no conjunto B = {m + n : (m, n) \em A}?",3,69,72
3,3Q1,"Um tanque contém inicialmente uma solução salina de 3 gramas de sal dissolvido em 100 litros de água. Uma solução salina contendo 0,02 gramas de sal por litro de água é pulverizada no tanque a uma taxa de 4 litros por minuto. A solução pulverizada é continuamente misturada com a solução salina no tanque, e a mistura flui para fora do tanque a uma taxa de 4 litros por minuto. Se a mistura for instantânea, quantos gramas de sal estão no tanque após 100 minutos?",3,165,168
4,4Q1,Um total de 30 jogadores jogarão basquete em um parque. Haverá exatamente 5 jogadores em cada time. Qual afirmação explica corretamente como encontrar o número de times necessários?,3,63,66


In [ ]:
results do figurative fazer os loads deles de novo 

In [18]:
df_fig_results = pd.read_csv(r"C:\Users\annap\Documents\UFLA\EVAL\DATASETS\mmlu_pro_to_gloss.csv")
df_fig_results.query("ID =='56Q1'")

,ID,PT,TOKENS_EST_ID,TOKENS_EST_PT,TOTAL_TOKENS
25,56Q1,"Você me fará a justiça de lembrar que sempre apoiei arduamente o Direito de cada homem à sua própria opinião, por mais diferente que essa opinião possa ser da minha. Aquele que nega a outro esse direito, torna-se prisioneiro de sua própria crença, porque se impede do direito de mudá-la.",3,110,113
26,56Q1,A ferramenta mais eficaz contra erros de todo tipo é a Razão. Nunca usei nenhuma outra e confio que nunca usarei.,3,42,45
27,56Q1,"A circunstância que agora ocorreu na França da abolição total de toda a ordem nacional do sacerdócio e de tudo o que pertence a sistemas compulsivos de religião e artigos compulsivos de fé não apenas precipitou minha intenção, mas tornou uma obra desse tipo extremamente necessária, para que no colapso geral da superstição, de falsos sistemas de governo e falsa teologia, não percamos de vista a moralidade, a humanidade e a teologia que é verdadeira.",3,161,164
28,56Q1,Minha própria consciência é minha própria guia.,3,21,24
29,56Q1,"Todas as instituições nacionais de igrejas, sejam judaicas, cristãs ou turcas, me parecem nada mais do que criações humanas, feitas para oprimir e controlar a humanidade e acumular poder e riqueza.",3,77,80


In [ ]:
pegar o resultado do figurative

In [1]:
figurative.query("ID =='56C1'")

NameError: name 'figurative' is not defined

In [17]:
df_glosa.query("ID =='56Q1'")

,ID,PT,GLOSA
25,56Q1,"Você me fará a justiça de lembrar que sempre apoiei arduamente o Direito de cada homem à sua própria opinião, por mais diferente que essa opinião possa ser da minha. Aquele que nega a outro esse direito, torna-se prisioneiro de sua própria crença, porque se impede do direito de mudá-la.",VOCÊ FAZER JUSTIÇA LEMBRAR EU SEMPRE APOIAR DIREITO HOMEM OPINIÃO DIFERENTE MINHA. AQUELE NEGAR OUTRO DIREITO PRISIONEIRO CRENÇA IMPEDIR DIREITO MUDAR.
26,56Q1,A ferramenta mais eficaz contra erros de todo tipo é a Razão. Nunca usei nenhuma outra e confio que nunca usarei.,FERRAMENTA EFICAZ ERRO RAZÃO. EU NUNCA USAR OUTRA CONFIAR NUNCA USAR.
27,56Q1,"A circunstância que agora ocorreu na França da abolição total de toda a ordem nacional do sacerdócio e de tudo o que pertence a sistemas compulsivos de religião e artigos compulsivos de fé não apenas precipitou minha intenção, mas tornou uma obra desse tipo extremamente necessária, para que no colapso geral da superstição, de falsos sistemas de governo e falsa teologia, não percamos de vista a moralidade, a humanidade e a teologia que é verdadeira.",CIRCUNSTÂNCIA FRANÇA ABOLIÇÃO ORDEM NACIONAL SACERDÓCIO PERTENCER SISTEMA COMPULSIVO RELIGIÃO ARTIGO COMPULSIVO FÉ PRECIPITAR INTENÇÃO OBRA NECESSÁRIO COLAPSO SUPERSITÇÃO SISTEMA GOVERNO FALSO TEOLOGIA FALSO PERDER MORALIDADE HUMANIDADE TEOLOGIA VERDADEIRO.
28,56Q1,Minha própria consciência é minha própria guia.,CONSCIÊNCIA GUIA.
29,56Q1,"Todas as instituições nacionais de igrejas, sejam judaicas, cristãs ou turcas, me parecem nada mais do que criações humanas, feitas para oprimir e controlar a humanidade e acumular poder e riqueza.",INSTITUIÇÃO NACIONAL IGREJA JUDAICA CRISTÃ TURCA CRIAÇÃO HUMANA OPRIMIR CONTROLAR HUMANIDADE ACUMULAR PODER RIQUEZA.


In [10]:
df_unicos = df_glosa.drop_duplicates(subset='ID', keep='first')
df_duplicados = df_glosa[df_glosa.duplicated(subset='ID', keep=False)]
ids_duplicados = df_glosa['ID'][df_glosa['ID'].duplicated()].unique()
df_glosa[df_glosa['ID'].isin(ids_duplicados)]

,ID,PT,GLOSA
25,56Q1,"Você me fará a justiça de lembrar que sempre apoiei arduamente o Direito de cada homem à sua própria opinião, por mais diferente que essa opinião possa ser da minha. Aquele que nega a outro esse direito, torna-se prisioneiro de sua própria crença, porque se impede do direito de mudá-la.",VOCÊ FAZER JUSTIÇA LEMBRAR EU SEMPRE APOIAR DIREITO HOMEM OPINIÃO DIFERENTE MINHA. AQUELE NEGAR OUTRO DIREITO PRISIONEIRO CRENÇA IMPEDIR DIREITO MUDAR.
26,56Q1,A ferramenta mais eficaz contra erros de todo tipo é a Razão. Nunca usei nenhuma outra e confio que nunca usarei.,FERRAMENTA EFICAZ ERRO RAZÃO. EU NUNCA USAR OUTRA CONFIAR NUNCA USAR.
27,56Q1,"A circunstância que agora ocorreu na França da abolição total de toda a ordem nacional do sacerdócio e de tudo o que pertence a sistemas compulsivos de religião e artigos compulsivos de fé não apenas precipitou minha intenção, mas tornou uma obra desse tipo extremamente necessária, para que no colapso geral da superstição, de falsos sistemas de governo e falsa teologia, não percamos de vista a moralidade, a humanidade e a teologia que é verdadeira.",CIRCUNSTÂNCIA FRANÇA ABOLIÇÃO ORDEM NACIONAL SACERDÓCIO PERTENCER SISTEMA COMPULSIVO RELIGIÃO ARTIGO COMPULSIVO FÉ PRECIPITAR INTENÇÃO OBRA NECESSÁRIO COLAPSO SUPERSITÇÃO SISTEMA GOVERNO FALSO TEOLOGIA FALSO PERDER MORALIDADE HUMANIDADE TEOLOGIA VERDADEIRO.
28,56Q1,Minha própria consciência é minha própria guia.,CONSCIÊNCIA GUIA.
29,56Q1,"Todas as instituições nacionais de igrejas, sejam judaicas, cristãs ou turcas, me parecem nada mais do que criações humanas, feitas para oprimir e controlar a humanidade e acumular poder e riqueza.",INSTITUIÇÃO NACIONAL IGREJA JUDAICA CRISTÃ TURCA CRIAÇÃO HUMANA OPRIMIR CONTROLAR HUMANIDADE ACUMULAR PODER RIQUEZA.
...,...,...,...
119417,11991C10,"O fluxo (a) é irrotacional com uma vorticidade de 12, o fluxo (b) é com rotação com uma vorticidade de 0",FLUXO A IRROTACIONAL VORTICIDADE 12 FLUXO B ROTAÇÃO VORTICIDADE 0
471,8290Q1,"Cinco anos atrás, um computador que funcionava com a metade da velocidade de um computador atual custava o dobro do dinheiro. A relação velocidade/preço do computador atual é qual porcentagem da relação velocidade/preço do computador antigo?",CINCO ANO ATRÁS COMPUTADOR VELOCIDADE METADE COMPUTADOR ATUAL CUSTAR DOBRO DINHEIRO. COMPUTADOR ATUAL VELOCIDADE PREÇO RELAÇÃO COMPUTADOR ANTIGO VELOCIDADE PREÇO RELAÇÃO PORCENTAGEM QUAL?
472,8290Q1,"Cinco anos atrás, um computador que funcionava com a metade da velocidade de um computador atual custava duas vezes mais caro. A relação velocidade/preço do computador atual é qual porcentagem da relação velocidade/preço do computador antigo?",COMPUTADOR ANTIGO CINCO-ANO ATRÁS VELOCIDADE METADE ATUAL CUSTAR DUAS-VEZES CARO. ATUAL VELOCIDADE PREÇO PORCENTAGEM ANTIGO QUAL?
579,888C1,"Por um lado, o naturalismo baseia-se num conjunto de regras com abrangência universal e propósito claro e, por outro lado, o positivismo assenta num sistema jurídico estruturado e coerente que é criado pelos Estados à luz dos seus interesses e desejos.",NATURALISMO UM LADO BASEAR REGRA CONJUNTO ABRANGÊNCIA UNIVERSAL PROPÓSITO CLARO OUTRO LADO POSITIVISMO ASSENTAR SISTEMA JURÍDICO ESTRUTURADO COERENTE ESTADO CRIAR INTERESSE DESEJO.


In [ ]:
to_gloss_again = df_base[~df_base['ID'].isin(df_bucket['ID'].unique())]
to_gloss_again.shape

## Last

In [51]:
print("Last---")
content_bucket = read_file_txt("../DATASETS/figurative_last_mllu_to_figurative_single_response.txt")
count_ids(content_bucket)
df_last = processar_content_para_dataframe(content_bucket)
#df_single = drop_duplicates_by_count_col(df_single, 'RESULTADO')
df_last.head()

Last---
Total de IDs com sufixo 'Q1': 15
Total de IDs com sufixo 'C1': 15
Total de IDs com sufixo 'C2': 21
Total de IDs com sufixo 'C3': 20
Total de IDs com sufixo 'C4': 22
Total ids: 93
DataFrame - shape: (57, 5), IDs únicos: 57


,ID,ORIGINAL,FIGURATIVO,REESCRITO,RESULTADO
0,2466Q1,"Construa uma tabela verdade completa para o seguinte argumento. Então, usando a tabela verdade, determine se o argumento é válido ou inválido. Se o argumento for inválido, escolha uma opção que apresente um contraexemplo. (Pode haver outros contraexemplos também.)",None,None,"Construa uma tabela verdade completa para o seguinte argumento. Então, usando a tabela verdade, determine se o argumento é válido ou inválido. Se o argumento for inválido, escolha uma opção que apresente um contraexemplo. (Pode haver outros contraexemplos também.)"
1,2521Q1,"Use tabelas de verdade indiretas para determinar se cada conjunto de proposições é consistente. Se o conjunto for consistente, escolha uma opção com uma avaliação consistente. (Pode haver outras avaliações consistentes.)",consistente,não contraditório,"Use tabelas de verdade indiretas para determinar se cada conjunto de proposições é não contraditório. Se o conjunto for não contraditório, escolha uma opção com uma avaliação não contraditória. (Pode haver outras avaliações não contraditórias.)"
2,2540Q1,"Use tabelas de verdade indiretas para determinar se o argumento a seguir é válido. Se o argumento for inválido, escolha uma opção que apresente um contraexemplo. (Pode haver outros contraexemplos também.)","Se o argumento for inválido, escolha uma opção que apresente um contraexemplo.","Se o argumento for inválido, escolha uma opção que mostre um exemplo que o refute.","Use tabelas de verdade indiretas para determinar se o argumento a seguir é válido. Se o argumento for inválido, escolha uma opção que mostre um exemplo que o refute. (Pode haver outros exemplos que o refutem também.)"
3,4885Q1,"O Gravitron é um brinquedo de parque de diversões que parece um grande cilindro. As pessoas ficam dentro do cilindro contra a parede enquanto ele começa a girar. Eventualmente, ele está girando rápido o suficiente para que o chão possa ser removido sem que ninguém caia.",o chão possa ser removido sem que ninguém caia,"a força centrípeta seja suficiente para manter as pessoas pressionadas contra a parede, impedindo-as de cair","O Gravitron é um brinquedo de parque de diversões que parece um grande cilindro. As pessoas ficam dentro do cilindro contra a parede enquanto ele começa a girar. Eventualmente, ele está girando rápido o suficiente para que a força centrípeta seja suficiente para manter as pessoas pressionadas contra a parede, impedindo-as de cair."
4,6433Q1,"\""O Estado não tem nada a ver com os quartos da nação\"", é uma citação famosa deste ex-primeiro-ministro canadense.",quartos da nação,problemas internos ou assuntos privados dos cidadãos,"\""O Estado não tem nada a ver com os problemas internos ou assuntos privados dos cidadãos\"", é uma citação famosa deste ex-primeiro-ministro canadense."


# Join e filter the last 

In [52]:
df_joined = pd.concat([df_bucket, df_single, df_last], axis=0)
df_joined.shape[0] == (df_bucket.shape[0] + df_single.shape[0] + df_last.shape[0])

True

In [53]:
not_figuratived = df_base[~df_base['ID'].isin(df_joined['ID'])]
not_figuratived.shape

(742, 5)

In [40]:
#not_figuratived.to_csv("../DATASETS/last_mllu_to_figurative_single.csv", index=False)

# Format to Gloss

In [54]:
def calculate_tokens(df, columns, tokenizer_model="gpt2"):
    """
    Calcula o número de tokens para colunas especificadas em um DataFrame usando um tokenizer.

    Args:
        df (pd.DataFrame): DataFrame de entrada contendo os dados de texto.
        columns (list): Lista de colunas do DataFrame para calcular os tokens.
        tokenizer_model (str): Nome do modelo para carregar o tokenizer. Padrão é "gpt2".

    Returns:
        pd.DataFrame: DataFrame atualizado com colunas contendo o número de tokens.
    """
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)

    def count_tokens(text):
        if not text or not isinstance(text, str):
            return 0
        return len(tokenizer.encode(text, truncation=True))

    for column in columns:
        token_column_name = f"TOKENS_EST_{column.upper()}"
        df[token_column_name] = df[column].apply(count_tokens)

    total_tokens_column = "TOTAL_TOKENS"
    token_columns = [f"TOKENS_EST_{col.upper()}" for col in columns if f"TOKENS_EST_{col.upper()}" in df.columns]

    if token_columns:
        df[total_tokens_column] = df[token_columns].sum(axis=1)
    else:
        df[total_tokens_column] = 0
    return df

In [55]:
df_to_gloss = df_joined[['ID', 'RESULTADO']].rename(columns={'RESULTADO':'PT'})
df_to_gloss.head()

,ID,PT
0,0Q1,"Encontre o grau para a extensão de campo dada Q(sqrt(2), sqrt(3), sqrt(18)) sobre Q."
1,1Q1,"Seja p = (1, 2, 5, 4)(2, 3) em S_5 . Encontre o índice de <p> em S_5."
2,2Q1,Encontre todos os zeros no campo finito indicado do polinômio dado com coeficientes nesse campo. x^5 + 3x^3 + x^2 + 2x em Z_5
3,3Q1,"Afirmação 1 | Um grupo de fatores de um grupo não-Abeliano é não-Abeliano. Afirmação 2 | Se K é um subgrupo normal de H e H é um subgrupo normal de G, então K é um subgrupo normal de G."
4,4Q1,"Encontre o produto dos polinômios dados no anel polinomial dado. f(x) = 4x - 5, g(x) = 2x^2 - 4x + 2 em Z_8[x]."


In [56]:
df_to_gloss_final = calculate_tokens(df_to_gloss, ['ID', 'PT'])
print(f"To gloss:{df_to_gloss_final.shape}, ids únicos: {df_to_gloss_final['ID'].nunique()}")
df_to_gloss_final.head()

To gloss:(77176, 5), ids únicos: 77084


,ID,PT,TOKENS_EST_ID,TOKENS_EST_PT,TOTAL_TOKENS
0,0Q1,"Encontre o grau para a extensão de campo dada Q(sqrt(2), sqrt(3), sqrt(18)) sobre Q.",3,37,40
1,1Q1,"Seja p = (1, 2, 5, 4)(2, 3) em S_5 . Encontre o índice de <p> em S_5.",3,39,42
2,2Q1,Encontre todos os zeros no campo finito indicado do polinômio dado com coeficientes nesse campo. x^5 + 3x^3 + x^2 + 2x em Z_5,3,52,55
3,3Q1,"Afirmação 1 | Um grupo de fatores de um grupo não-Abeliano é não-Abeliano. Afirmação 2 | Se K é um subgrupo normal de H e H é um subgrupo normal de G, então K é um subgrupo normal de G.",3,71,74
4,4Q1,"Encontre o produto dos polinômios dados no anel polinomial dado. f(x) = 4x - 5, g(x) = 2x^2 - 4x + 2 em Z_8[x].",3,54,57


In [57]:
df_to_gloss_final.to_csv("../DATASETS/mmlu_to_gloss.csv", index=False)

# Process to inference

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/home/annap/Documents/chatbot-project/EVAL/DATASETS/MMLU/07_mmlu_pre_processed.csv")

In [3]:
df.head()

,ID,Q1,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,question_id,subject,is_duplicate
0,0,GRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA ...,ZERO,4,DOIS.,SEIS,NaN,NaN,NaN,NaN,NaN,NaN,0,abstract_algebra,False
1,1,P 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.,OITO,2,VINTE QUATRO.,CENTO E VINTE,NaN,NaN,NaN,NaN,NaN,NaN,1,abstract_algebra,False
2,2,ZERO TODOS ENCONTRAR CAMPO FINITO INDICADO POL...,ZERO,1,ZERO VÍRGULA UM.,ZERO VÍRGULA QUATRO,NaN,NaN,NaN,NaN,NaN,NaN,2,abstract_algebra,False
3,3,AFIRMAÇÃO 1 GRUPO FATORES GRUPO NÃO-ABELIANO N...,VERDADE FATO.,MENTIROSO MENTIROSO,"VERDADEIRO, FALSO.",FALSO VERDADEIRO,NaN,NaN,NaN,NaN,NaN,NaN,3,abstract_algebra,False
4,4,PRODUTO ENCONTRAR POLINÔMIO ANEL POLINÔMIO F X...,DOIS X QUADRADO MAIS CINCO,6X^2 4X 6 +,ZERO.,X DOIS MAIS UM,NaN,NaN,NaN,NaN,NaN,NaN,4,abstract_algebra,False
